In [0]:
#Claves del Datalake
storage_account_name = "plazrdlsdwhscusdevtest"
blob_container_name = "dwhdevblob"
key =r"Zx7Rg4HPYFsxjFfnELaTlyd7uT07gwuRqO6ObxdtA1LUcvqn/a9UJPElu6qQrAkhvoNBBVdoSW3c+9wmi3WQYw=="

#
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".dfs.core.windows.net",
  key)
##

file_type = "parquet" 

In [0]:
#Funcion de Save en Databricks
def Save_Table_Sustituir(df,DataSave,NameTable,Path):
  
  #Carpeta donde se guarda
  DELTALAKE_SILVER_PATH_Datos = Path + NameTable
  # Remove folder if it exists
  dbutils.fs.rm(DELTALAKE_SILVER_PATH_Datos, recurse=True)
  #Se guarda en formato delta
  df.write.format("delta").save(DELTALAKE_SILVER_PATH_Datos)

  D0 = str(DataSave)
  F0 = D0 + "." + NameTable
  F0 = str(F0)
  
  QUERY_0 = (f"""CREATE DATABASE IF NOT EXISTS {D0};""")
  spark.sql(QUERY_0)

  QUERY_1 = (f"""DROP TABLE IF EXISTS {F0};""")
  spark.sql(QUERY_1)

  QUERY_2 = (f"""CREATE TABLE {F0}
           USING delta
           LOCATION '{DELTALAKE_SILVER_PATH_Datos}';
           """)
  spark.sql(QUERY_2)

  print("Tabla en BD:",F0)
     
  return F0

In [0]:
# LIMPEZA DE DATOS
from pyspark.sql.functions import *
import quinn
import numpy as np
import pandas as pd

#Renomabrar Columnas Funcion
def rename_columns(df, columns):
    if isinstance(columns, dict):
        for old_name, new_name in columns.items():
            df = df.withColumnRenamed(old_name, new_name)
        return df
    else:
        raise ValueError("'columns' should be a dict, like {'old_name_1':'new_name_1', 'old_name_2':'new_name_2'}")
        
def limpieza_Datos(DF):
  
  #from pyspark.sql.functions import *
  
  for C, S in DF.dtypes:
    #Quita Espacios en Blanco antes y despues
    DF = DF.withColumn(C, trim(col(C)))    # trim es sólo para strings, por eso los vuelve strings 
    #Vuelve parrafos a un espacio simple
    DF = DF.withColumn(C, quinn.single_space(col(C)))
    if S != 'string':
      DF = DF.withColumn(C, DF[C].cast(S))
      
    # Se quita en la columna el T$ 
    U = C.replace('T$', '')
    U = str(U)
    DF = rename_columns(DF, {C : U})
    
  return DF  
    
  #return df

In [0]:
# setting the path for each table
path_1 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTCCOM010007/data_b35741b5-3c3d-4ab2-9f81-19272b4248cd_f6713763-60f6-4ef9-a752-a48672129760.parquet"
path_2 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDPGE808006/PGE_TAM.TTDPGE808006.parquet"
path_3 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDPGE940006/data_07bf1223-b0d8-453d-84fe-df77f5784e83_67299770-28b6-4bdb-935e-d0ab24b4c1fc.parquet"
path_4 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDPGE941006/data_29f3db68-40b7-4c82-8b2e-90f192185638_8c08d908-0cc5-4d89-859a-9ffcfd7a0188.parquet"
path_5 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDSLS040006/data_075415c2-9baa-4488-a52e-2fe92bd05ee4_10596be4-b8df-4f1e-9340-e903651ff1ef.parquet"
path_6 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDSLS041006/data_f9b4f72e-de79-4e34-90e2-5cc5a52b130b_b9cab09d-4441-45fb-8550-eae415cb7a66.parquet"
path_7 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTDSLS045006/data_19135094-4b7f-4692-b0dd-797826ce8d43_3fad2579-6eef-453f-b624-84980e5591fd.parquet"
path_8 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTIITM001006/data_22a0fb2e-02ec-45c7-bdbd-50eb86f1c78d_21d98fc0-f76f-4681-aea2-7eaa9f947271.parquet"
path_9 = "SandBox/DataAnalytics/jaime_figueroa/TAM/RawData/TTIPGE001006/data_f52e4dd2-d69c-4cf2-b56e-c9b895d95ac2_2570abd3-37bf-4fdc-ae2c-d9871f371693.parquet"

In [0]:
# setting the location of each data frame
file_location_1 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_1)
file_location_2 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_2)
file_location_3 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_3)
file_location_4 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_4)
file_location_5 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_5)
file_location_6 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_6)
file_location_7 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_7)
file_location_8 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_8)
file_location_9 = "abfss://%s@%s.dfs.core.windows.net/%s" % (blob_container_name, storage_account_name, path_9)

In [0]:
# loading in the data frames
df_1 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_1)
df_2 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_2)
df_3 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_3)
df_4 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_4)
df_5 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_5)
df_6 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_6)
df_7 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_7)
df_8 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_8)
df_9 = spark.read.format(file_type).options(header='true', inferschema='true').load(file_location_9)

In [0]:
# debo quitar las T$ de cada columna de todas las tablas y convertirlas en sql tables
#infoCliente = df_1.toDF(*[col[2:] for col in df_1.columns])
#mercado = df_2.toDF(*[col[2:] for col in df_2.columns])
#ordenVentaComp = df_3.toDF(*[col[2:] for col in df_3.columns])
#lineaOrdenVentaComp = df_4.toDF(*[col[2:] for col in df_4.columns])
#ordenVenta = df_5.toDF(*[col[2:] for col in df_5.columns])
#lineaOrdenVenta = df_6.toDF(*[col[2:] for col in df_6.columns])
#factura = df_7.toDF(*[col[2:] for col in df_7.columns])
#item = df_8.toDF(*[col[2:] for col in df_8.columns])
#itemComplemento = df_9.toDF(*[col[2:] for col in df_9.columns])

In [0]:
# nombrar las tablas
item = limpieza_Datos(df_8)
itemComplemento = limpieza_Datos(df_9)
factura = limpieza_Datos(df_7)
mercado = limpieza_Datos(df_2)
infoCliente = limpieza_Datos(df_1)
ordenVenta = limpieza_Datos(df_5)
lineaOrdenVenta = limpieza_Datos(df_6)
lineaOrdenVentaComp = limpieza_Datos(df_4)
ordenVentaComp = limpieza_Datos(df_3)

In [0]:
# guardar las tablas
Save_Table_Sustituir(item,"DataSet","item","/mnt/delta/drvdev/")
Save_Table_Sustituir(itemComplemento,"DataSet","itemComplemento","/mnt/delta/drvdev/")
Save_Table_Sustituir(factura,"DataSet","factura","/mnt/delta/drvdev/")
Save_Table_Sustituir(mercado,"DataSet","mercado","/mnt/delta/drvdev/")
Save_Table_Sustituir(infoCliente,"DataSet","infoCliente","/mnt/delta/drvdev/")
Save_Table_Sustituir(ordenVenta,"DataSet","ordenVenta","/mnt/delta/drvdev/")
Save_Table_Sustituir(lineaOrdenVenta,"DataSet","lineaOrdenVenta","/mnt/delta/drvdev/")
Save_Table_Sustituir(lineaOrdenVentaComp,"DataSet","lineaOrdenVentaComp","/mnt/delta/drvdev/")
Save_Table_Sustituir(ordenVentaComp,"DataSet","ordenVentaComp","/mnt/delta/drvdev/")

Tabla en BD: DataSet.item
Tabla en BD: DataSet.itemComplemento
Tabla en BD: DataSet.factura
Tabla en BD: DataSet.mercado
Tabla en BD: DataSet.infoCliente
Tabla en BD: DataSet.ordenVenta
Tabla en BD: DataSet.lineaOrdenVenta
Tabla en BD: DataSet.lineaOrdenVentaComp
Tabla en BD: DataSet.ordenVentaComp
Out[9]: 'DataSet.ordenVentaComp'